In [1]:
### IMPORT PACKAGES --- PIP INSTALL ANY MISSING PACKAGES IN CMD LINE ---Written in PYTHON 3 (iPython Notebook) - PJK ###
import numpy as np
import pandas as pd 
import datetime as dt
import glob2, os
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.dates as mdates
import fnmatch
from datetime import timedelta
# interactive packages (test)
import plotly.plotly as py
from plotly.graph_objs import *
import bokeh
from bokeh.plotting import figure, show, output_file, vplot
from bokeh.models import LinearAxis, Range1d
import seaborn

ModuleNotFoundError: No module named 'glob2'

In [167]:
### RUN DIRECTORIES, FILE PARSING CONSTANTS ###

# Set the Run Directory #
runDir = "X:\\LRG_2015M\\OUT_TR\\20160523a\\CanutilloTesting_23a23b\\"

##Get Model File(s), check root directory for the names of the alternate scenario directories and append appropriately ##
BASE_GL = glob2.glob("LRG_SGL_20160523a.drt")
BASE_SW = glob2.glob("LRG_SSW_20160523a.drt")

ALT1_GL = glob2.glob("LRG_SGL_20160523b.drt")
ALT1_SW = glob2.glob("LRG_SSW_20160523b.drt")

#ALT2_GL = glob2.glob("ETSW/*GL.drt")
#ALT2_SW = glob2.glob("ETSW/*SW.drt")

# File Parsing Offset Values, to skip ID info block --- CHECK THESE #
gloff = 19
swoff = 45
## number of rows to read for the ID info block at beginning of file --- CHECK THESE ##
swnrows = 38
glnrows = 12

In [168]:
### DATA IMPORT PROCEDURES ###

## Get the header and ID Descriptions from the model output files, CREATE DFs, Store in List, Merge with ID List ##

BASE_ID_List = {}
BASE_List = {}

for file in BASE_GL:
    BASE_ID_List[file] = pd.read_fwf(file,skiprows = [0,1,3], header=1, nrows= glnrows ,colspecs=[(1,5),(6,38)])
    BASE_ID_List[file].ID = pd.to_numeric(BASE_ID_List[file].ID, errors='coerce')
    
    BASE_List[file] = pd.read_fwf(file, skiprows= gloff, header=0,colspecs=[(1,5),(6,10),(11,20),(21,25),(26,40)])
    BASE_List[file] = BASE_List[file].merge(BASE_ID_List[file], on='ID')
    
for file in BASE_SW:
    BASE_ID_List[file] = pd.read_fwf(file, skiprows= [0,1,3], header=1, nrows= swnrows,colspecs=[(1,5),(6,38)])
    BASE_ID_List[file].ID = pd.to_numeric(BASE_ID_List[file].ID, errors='coerce')
    
    BASE_List[file] = pd.read_fwf(file, skiprows= swoff, header=0,colspecs=[(1,5),(6,10),(11,20),(21,25),(26,40)])
    BASE_List[file] = BASE_List[file].merge(BASE_ID_List[file], on='ID')
    
ALT1_ID_List = {}
ALT1_List = {}

for file in ALT1_GL:
    ALT1_ID_List[file] = pd.read_fwf(file, skiprows= [0,1,3], header=1, nrows= glnrows,colspecs=[(1,5),(6,38)])
    ALT1_ID_List[file].ID = pd.to_numeric(ALT1_ID_List[file].ID, errors='coerce')
    
    ALT1_List[file] = pd.read_fwf(file, skiprows= gloff, header=0,colspecs=[(1,5),(6,10),(11,20),(21,25),(26,40)])
    ALT1_List[file] = ALT1_List[file].merge(ALT1_ID_List[file], on='ID')
    
for file in ALT1_SW:
    ALT1_ID_List[file] = pd.read_fwf(file, skiprows= [0,1,3], header=1, nrows= swnrows,colspecs=[(1,5),(6,38)])
    ALT1_ID_List[file].ID = pd.to_numeric(ALT1_ID_List[file].ID, errors='coerce')
    
    ALT1_List[file] = pd.read_fwf(file, skiprows= swoff, header=0,colspecs=[(1,5),(6,10),(11,20),(21,25),(26,40)])
    ALT1_List[file] = ALT1_List[file].merge(ALT1_ID_List[file], on='ID')  

In [172]:
## INTERACTIVE PLOTTING BLOCK --- SW FILES 
i = 0
p = []

#### NAME HTML FILE ####
output_file("SW_20160523a-b.html", title="InteractivePLotting-SW.py")

## Create a Reference DF with end dates for each stress period based on a BASE case file ##
startdate = dt.date(1940,2,29)
cum_day = BASE_List[BASE_GL[0]][['SPER','TOTALDAY']].drop_duplicates()
cum_day['cumday'] = cum_day.loc[:,('TOTALDAY')].cumsum()
cum_day['date'] = cum_day['cumday'].apply(lambda x: startdate + dt.timedelta(days=x))

for fileB, fileA in zip(BASE_SW, ALT1_SW):
    
    #IDs_BASE_SW = pd.unique(BASE_List[fileB].DESCRIPTION.ravel())
    IDs_BASE_SW = ['Rio Grande below Mesilla', 'Rio Grande at Anthony',
       'Rio Grande at El Paso', 'Del Rio Drain',
       'La Mesa Drain','East Drain',
       'Montoya Drain', 'La Union East Stateline',
       'La Union West Stateline', '3 Saints East Stateline',
       'East Drain Stateline', 'Anthony Drain Stateline',
       'West Drain Sateline', 'Anthony Drain',
       'West Drain', 'Nemexas Drain']
    
    #IDs_ALT1_SW = pd.unique(ALT1_List[fileA].DESCRIPTION.ravel())
    IDs_ALT1_SW = ['Rio Grande below Mesilla', 'Rio Grande at Anthony',
       'Rio Grande at El Paso', 'Del Rio Drain',
       'La Mesa Drain','East Drain',
       'Montoya Drain', 'La Union East Stateline',
       'La Union West Stateline', '3 Saints East Stateline',
       'East Drain Stateline', 'Anthony Drain Stateline',
       'West Drain Sateline', 'Anthony Drain',
       'West Drain', 'Nemexas Drain']
      
    # Generate dynamic plot counts
    #nb_plotsSW = IDs_BASE_SW.shape[0]
    nb_plotsSW = len(IDs_BASE_SW)
    plots = list(range(0,nb_plotsSW))
    
    if fileB == BASE_SW[0] and fileA == ALT1_SW[0] :
        
        for locB,locA in zip (IDs_BASE_SW,IDs_ALT1_SW):
            plots[i] = figure(x_axis_type = "datetime", plot_width=1800, plot_height=600)
            subsetB = BASE_List[fileB][BASE_List[fileB]["DESCRIPTION"].isin([locB])]
            subsetA = ALT1_List[fileA][ALT1_List[fileA]["DESCRIPTION"].isin([locA])]
            subsetA.loc[:,('Diff')] = subsetA['FLUX'].subtract(subsetB['FLUX'], fill_value=0)
            
            ## merge the subset with the end date DF calc'ed above ##
            subsetB = subsetB.merge(cum_day)
            subsetA = subsetA.merge(cum_day)
            
            # Select Min/Max # 
            y2min = subsetA['Diff'].min()
            y2max = subsetA['Diff'].max()
            
            ## SECONDARY Y-AXIS & PLOTTING ##
            plots[i].line(np.array(subsetB['date']), subsetB['FLUX'],line_width=2, color='#ef4338', legend='Base Case: ' + BASE_SW[0])
            plots[i].line(np.array(subsetA['date']), subsetA['FLUX'],line_width=2, color='#58c0ff', legend='Perturbation: ' + ALT1_SW[0])

            plots[i].extra_y_ranges = {"diff": Range1d(start=y2min, end=y2max)}
            plots[i].line(np.array(subsetA['date']), subsetA['Diff'],line_width=2,line_dash=[4, 4], color='#60b436', legend='Difference',y_range_name="diff")
            plots[i].add_layout(LinearAxis(y_range_name="diff"), 'right')

            ## Figure Formatting ##
            plots[i].title = locB
            plots[i].background_fill_color = "whitesmoke"
            plots[i].grid.grid_line_alpha=0.6
            plots[i].xaxis.axis_label = 'Date'
            plots[i].yaxis.axis_label = 'AFY'
            
            p.append(plots[i])
            i = i + 1
            
lst_p = vplot(*p)
show(lst_p)


C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:266: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:426: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

ERROR:C:\Anaconda3\lib\site-packages\bokeh\core\validation\check.py:W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure, ViewModel:Plot, ref _id: 8626e4af-f7ea-4aed-9f33-4844213ebe68
ERROR:C:\Anaconda3\lib\site-packages\bokeh\core\validation\check.py:W-1002 (EMPTY_LAYOUT): Layout has no children: VBox, ViewModel:VBox, ref

In [174]:
## INTERACTIVE PLOTTING BLOCK --- GL FILES ## NOTE: THIS VERSION USES A HARDCODED number of plots per file
i = 0
p = []

#### NAME HTML FILE ####
output_file("GL_20160523a-b.html", title="InteractivePLotting-GL.py")

for fileB1, fileA1 in zip(BASE_GL, ALT1_GL):

    IDs_BASE_GL = pd.unique(BASE_List[fileB1].DESCRIPTION.ravel())
    IDs_ALT1_GL = pd.unique(ALT1_List[fileA1].DESCRIPTION.ravel())
        
    # Generate dynamic plot counts
    nb_plotsGL = IDs_BASE_GL.shape[0]
    plots = list(range(0,nb_plotsGL))
    
    if fileB1 == BASE_GL[0] and fileA1 == ALT1_GL[0] :
        
        for locB1,locA1 in zip (IDs_BASE_GL,IDs_ALT1_GL):
            
            plots[i] = figure(x_axis_type = "datetime", plot_width=1800, plot_height=600)
            subsetB = BASE_List[fileB1][BASE_List[fileB1]["DESCRIPTION"].isin([locB1])]
            subsetA = ALT1_List[fileA1][ALT1_List[fileA1]["DESCRIPTION"].isin([locA1])]
            subsetA.loc[:,('Diff')] = subsetA['FLUX'].subtract(subsetB['FLUX'], fill_value=0)
            
            ## merge the subset with the end date DF calc'ed above ##
            subsetB = subsetB.merge(cum_day)
            subsetA = subsetA.merge(cum_day)
            
            # Select Min/Max # 
            y2min = subsetA['Diff'].min()
            y2max = subsetA['Diff'].max()
            
            ## SECONDARY Y-AXIS & PLOTTING ##
            plots[i].line(np.array(subsetB['date']), subsetB['FLUX'],line_width=2, color='#ef4338', legend='Base Case: ' + BASE_GL[0])
            plots[i].line(np.array(subsetA['date']), subsetA['FLUX'],line_width=2, color='#58c0ff', legend='Perturbation: ' + ALT1_GL[0])

            plots[i].extra_y_ranges = {"diff": Range1d(start=y2min, end=y2max)}
            plots[i].line(np.array(subsetA['date']), subsetA['Diff'],line_width=2,line_dash=[4, 4], color='#60b436', legend='Difference:',y_range_name="diff")
            plots[i].add_layout(LinearAxis(y_range_name="diff"), 'right')

            ## Figure Formatting ##
            plots[i].title = locB1
            plots[i].background_fill_color = "whitesmoke"
            plots[i].grid.grid_line_alpha=0.6
            plots[i].xaxis.axis_label = 'Date'
            plots[i].yaxis.axis_label = 'AFY'
            
            p.append(plots[i])
            i = i + 1
            
lst_p = vplot(*p)
show(lst_p)


C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:266: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:426: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

ERROR:C:\Anaconda3\lib\site-packages\bokeh\core\validation\check.py:W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure, ViewModel:Plot, ref _id: 8626e4af-f7ea-4aed-9f33-4844213ebe68
ERROR:C:\Anaconda3\lib\site-packages\bokeh\core\validation\check.py:W-1002 (EMPTY_LAYOUT): Layout has no children: VBox, ViewModel:VBox, ref